# クローリング

## 準備

In [1]:
from selenium import webdriver

dir_v = "data"
br = webdriver.Chrome()

## コード一覧取得

In [1]:
from data_operation import DataOperation

In [2]:
op = DataOperation()

In [3]:
code_list = op.get_stock_list(code_from=9000)

試す

In [4]:
code_list[5]["code"]

9008

In [5]:
for code in code_list:
    print(code["code"])

9001
9003
9005
9006
9007
9008
9009
9010
9012
9014
9017
9020
9021
9022
9027
9028
9029
9031
9033
9034
9035
9036
9037
9039
9040
9041
9042
9044
9045
9046
9047
9048
9049
9051
9052
9055
9056
9057
9058
9059
9060
9062
9063
9064
9065
9066
9067
9068
9069
9070
9072
9073
9074
9075
9076
9077
9078
9081
9082
9083
9085
9086
9087
9101
9104
9107
9110
9113
9115
9119
9127
9130
9132
9133
9171
9173
9176
9179
9180
9193
9202
9204
9232
9233
9234
9301
9302
9303
9304
9305
9306
9307
9308
9310
9311
9312
9313
9317
9318
9319
9320
9322
9324
9351
9353
9355
9357
9358
9359
9360
9361
9362
9363
9364
9365
9366
9367
9368
9369
9370
9373
9375
9376
9377
9378
9380
9381
9382
9384
9385
9399
9401
9402
9404
9405
9407
9408
9409
9412
9413
9421
9422
9423
9424
9425
9427
9428
9430
9432
9433
9435
9436
9437
9438
9439
9441
9444
9445
9446
9449
9470
9471
9474
9475
9476
9477
9478
9479
9481
9501
9502
9503
9504
9505
9506
9507
9508
9509
9511
9513
9514
9531
9532
9533
9534
9535
9536
9537
9539
9540
9541
9542
9543
9544
9600
9601
9602
9603
9605
9607


In [11]:
op.db.sql="select code from stock.stock_list where code>2000 and code <3000 and status='defaut' order by code asc;"

In [14]:
a="aaaa{codea}bbb{codeb}"

In [15]:
a

'aaaa{codea}bbb{codeb}'

In [17]:
a.format(codea=111, codeb=222)

'aaaa111bbb222'

In [2]:
for a in range(1,3):
    print(a)

1
2


In [12]:
stock_list = op.db.execute()

In [13]:
stock_list

[{'code': 2001},
 {'code': 2002},
 {'code': 2003},
 {'code': 2004},
 {'code': 2006},
 {'code': 2008},
 {'code': 2009},
 {'code': 2052},
 {'code': 2053},
 {'code': 2055},
 {'code': 2056},
 {'code': 2058},
 {'code': 2107},
 {'code': 2108},
 {'code': 2109},
 {'code': 2112},
 {'code': 2114},
 {'code': 2117},
 {'code': 2120},
 {'code': 2121},
 {'code': 2122},
 {'code': 2124},
 {'code': 2127},
 {'code': 2128},
 {'code': 2130},
 {'code': 2131},
 {'code': 2132},
 {'code': 2134},
 {'code': 2136},
 {'code': 2137},
 {'code': 2138},
 {'code': 2139},
 {'code': 2140},
 {'code': 2144},
 {'code': 2145},
 {'code': 2146},
 {'code': 2148},
 {'code': 2150},
 {'code': 2151},
 {'code': 2152},
 {'code': 2153},
 {'code': 2154},
 {'code': 2155},
 {'code': 2156},
 {'code': 2157},
 {'code': 2158},
 {'code': 2159},
 {'code': 2160},
 {'code': 2162},
 {'code': 2163},
 {'code': 2164},
 {'code': 2165},
 {'code': 2166},
 {'code': 2168},
 {'code': 2169},
 {'code': 2170},
 {'code': 2172},
 {'code': 2173},
 {'code': 2174

In [6]:
import datetime

In [8]:
today = datetime.date.today()

In [10]:
today.year

2017

## 株価取得

In [8]:
def crawling_price(code):
    url="https://minkabu.jp/stock/{}"
    br.get(url.format(code))
    time.sleep(3)
    price_locator = "//*[@class='stock_price']"
    per_locator = "//th[contains(text(),'PER')]/following-sibling::*[contains(text(),'倍')]"
    date_locator = "//h2[@class='stock_label'][contains(text(),'株価')]/parent::*"
    price = br.find_element_by_xpath(price_locator).text.replace(",","")
    try:
        per = br.find_element_by_xpath(per_locator).text[:-1]
    except:
        per = "null"
    date_pre = br.find_element_by_xpath(date_locator).text
    date = date_pre[date_pre.find("(")+1:date_pre.find(")")]
    return {"code":code, "price":price, "per":per, "date":date}

In [9]:
crawling_price(1331)

{'code': 1331, 'date': '09/21', 'per': 'null', 'price': '159.0'}

## 業績取得

In [5]:
import re

In [32]:
id = "{}{}".format(code, year)

In [6]:
def crawling_financial_data(code):
    url="http://ke.kabupro.jp/xbrl/{}.htm"
    br.get(url.format(code))
    
    def get_years(text):
        search_re = re.search(r"\d{4}", text)
        return text[search_re.start():search_re.end()]
    
    xpath = "//*[contains(@class,'CellName')]//*[contains(text(),'通期')]"

    texts = list(map(lambda el: el.text, br.find_elements_by_xpath(xpath)))
    years = list(map(get_years, texts))
    
    xpath_v = xpath + "/parent::*/following-sibling::*[{}]"
    
    sales = list(map(lambda el: el.text, br.find_elements_by_xpath(xpath_v.format(1))))
    operating_profits = list(map(lambda el: el.text, br.find_elements_by_xpath(xpath_v.format(2))))
    net_incomes = list(map(lambda el: el.text, br.find_elements_by_xpath(xpath_v.format(3))))
    
    return {"code": code, "years": years, "sales": sales, "operating_profits": operating_profits, "net_incomes" : net_incomes}

In [7]:
crawling_financial_data(1301)

{'code': 1301,
 'net_incomes': ['2,100',
  '1,799',
  '2,433',
  '2,968',
  '1,269',
  '423',
  '58',
  '1,086',
  '1,587',
  '1,497'],
 'operating_profits': ['3,500',
  '2,433',
  '2,460',
  '2,915',
  '2,324',
  '1,636',
  '1,588',
  '2,180',
  '2,981',
  '2,926'],
 'sales': ['244,000',
  '226,626',
  '218,350',
  '202,387',
  '178,046',
  '181,885',
  '162,731',
  '145,778',
  '147,554',
  '147,767'],
 'years': ['2017',
  '2016',
  '2015',
  '2014',
  '2013',
  '2012',
  '2011',
  '2010',
  '2009',
  '2008']}

## 関数を試す

In [1]:
from crawling import Crawling

In [2]:
cr = Crawling(mode="debug")

In [3]:
cr.crawling_financial_data(1301)

{'code': 1301,
 'net_incomes': ['2,100',
  '1,799',
  '2,433',
  '2,968',
  '1,269',
  '423',
  '58',
  '1,086',
  '1,587',
  '1,497'],
 'operating_profits': ['3,500',
  '2,433',
  '2,460',
  '2,915',
  '2,324',
  '1,636',
  '1,588',
  '2,180',
  '2,981',
  '2,926'],
 'sales': ['244,000',
  '226,626',
  '218,350',
  '202,387',
  '178,046',
  '181,885',
  '162,731',
  '145,778',
  '147,554',
  '147,767'],
 'years': ['2017',
  '2016',
  '2015',
  '2014',
  '2013',
  '2012',
  '2011',
  '2010',
  '2009',
  '2008']}

In [4]:
cr.br.quit()

## 終了

In [8]:
br.quit()

# sql

In [1]:
from database_sql import Database

## 株価登録

In [10]:
import datetime

In [11]:
def set_price(code):
    db=Database()
    price = crawling_price(code)
    price["date"] = str(datetime.date.today().year)+"/"+price["date"]
    price["id"] = str(price["code"])+price["date"]
    db.sql="insert into stock.price (id, code, last, per, date) values ('{}', {}, {}, {}, '{}')".format( \
        price["id"],price["code"],price["price"],price["per"],price["date"])
    print(db.sql)
    db.execute()

In [5]:
db = Database()
db.sql="select * from stock.stock_list"
results=db.execute()

In [6]:
results

[{'code': 1301,
  'id': 3611,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1332,
  'id': 3612,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1334,
  'id': 3613,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1352,
  'id': 3614,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1376,
  'id': 3615,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1377,
  'id': 3616,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1378,
  'id': 3617,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1379,
  'id': 3618,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47, 24)},
 {'code': 1380,
  'id': 3619,
  'status': 'defaut',
  'update_at': datetime.datetime(2017, 10, 9, 20, 47

In [10]:
results[0]['date']

datetime.date(2017, 11, 5)

## 業績登録

insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(1, 1111, 1111, 1, 1, 1 );

In [7]:
db = Database()

In [10]:
from crawling import Crawling

In [46]:
def set_financial_data(data):
    db = Database()
    sql = "insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values({}, {}, {}, {}, {}, {});"
    for i in range(0, len(data["years"])):
        code = data["code"]
        year = data["years"][i]
        id = "{}{}".format(code, year)
        sales = data["sales"][i].replace(",","")
        operating_profit = data["operating_profits"][i].replace(",","")
        net_income = data["net_incomes"][i].replace(",","")
        
        db.sql = sql.format(id, code, year, sales, operating_profit, net_income)
        db.execute()

        print(db.sql)

In [47]:
set_financial_data(data)

insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012017, 1301, 2017, 244000, 3500, 2100);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012016, 1301, 2016, 226626, 2433, 1799);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012015, 1301, 2015, 218350, 2460, 2433);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012014, 1301, 2014, 202387, 2915, 2968);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012013, 1301, 2013, 178046, 2324, 1269);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012012, 1301, 2012, 181885, 1636, 423);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012011, 1301, 2011, 162731, 1588, 58);
insert into stock.financial_data (id, code, 

In [ ]:
cr = Crawling(mode="debug")

In [12]:
code=1301
data = cr.crawling_financial_data(code)

In [13]:
cr.br.quit()

In [28]:
data.keys()

dict_keys(['code', 'years', 'sales', 'operating_profits', 'net_incomes'])

In [31]:
sql = "insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values({}, {}, {}, {}, {}, {});"

In [24]:
sql.format(1,1,1,1,1,1)

'insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(1, 1, 1, 1, 1, 1);'

In [30]:
for i in range(0, len(data["years"])):
    code = data["code"]
    year = data["years"][i]
    id = "{}{}".format(code, year)
    sales = data["sales"][i]
    operating_profit = data["operating_profits"][i]
    net_income = data["net_incomes"][i]
    
    print(sql.format(id, code, year, sales, operating_profit, net_income))

insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012017, 1301, 2017, 244,000, 3,500, 2,100);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012016, 1301, 2016, 226,626, 2,433, 1,799);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012015, 1301, 2015, 218,350, 2,460, 2,433);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012014, 1301, 2014, 202,387, 2,915, 2,968);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012013, 1301, 2013, 178,046, 2,324, 1,269);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012012, 1301, 2012, 181,885, 1,636, 423);
insert into stock.financial_data (id, code, year, sales, operating_profit, net_income) values(13012011, 1301, 2011, 162,731, 1,588, 58);
insert into stock.financi

## いろいろ

In [19]:
str(datetime.date.today().year)+"/11/05"

'2017/11/05'

In [75]:
price = get_price_and_per(1401)

In [76]:
price

{'code': 1401, 'date': '11/02', 'per': '25.07', 'price': '696.0'}

In [77]:
price["date"] = str(datetime.date.today().year)+"/11/05"
price["id"] = str(price["code"])+price["date"]

In [79]:
db.sql="insert into stock.price (id, code, last, per, date) values ('{}', {}, {}, {}, '{}')".format(
    price["id"],price["code"],price["price"],price["per"],price["date"])

In [80]:
db.sql[:-1]

"insert into stock.price (id, code, last, per, date) values ('14012017/11/05', 1401, 696.0, 25.07, '2017/11/05'"

In [81]:
db.execute()

()

# 試す

In [ ]:
from data_operation import DataOperation

op = DataOperation()
op.db.sql="select * from stock.stock_list where code > 0;"
stock_list = op.db.execute()
codes=[]
for item in stock_list:
    codes.append(item["code"])

op.set_price(codes)